In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import torch

In [2]:
from datasets import load_dataset

data_down_dir = "/root/storage/nas/JH_server/2025/Synthetic_data/0_dataset"

dataset = load_dataset(f"{data_down_dir}/reuters_21578")
# 예시 출력
print(dataset)
print(dataset["train"][0])

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5485
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2189
    })
})
{'text': 'champion products ch approves stock split champion products inc said its board of directors approved a two for one stock split of its common shares for shareholders of record as of april the company also said its board voted to recommend to shareholders at the annual meeting april an increase in the authorized capital stock from five mln to mln shares reuter', 'label': 2}


In [3]:
from collections import defaultdict

label_counter = defaultdict(int)

for row in dataset['train']:
    label_counter[row['label']] += 1

In [8]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='/root/storage/nas/JH_server/cache/')

def preprocess(inputs):
    return tokenizer(inputs["text"], truncation=True, padding="max_length", max_length=512)

train_dataset = dataset["train"].map(preprocess, batched=False)
test_dataset = dataset["test"].map(preprocess, batched=False)

num_labels =  len(label_counter)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, cache_dir='/root/storage/nas/JH_server/cache/')

accuracy = evaluate.load("accuracy")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy.compute(predictions=preds, references=labels)

    return acc

args = TrainingArguments(
    eval_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    save_strategy="no"
)

trainer = Trainer(
    model = model,
    args=args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

                                    

Map:   0%|          | 0/5485 [00:00<?, ? examples/s]

Map:   0%|          | 0/2189 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_334040/4167039497.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy
100,No log,0.111876,0.973047
200,No log,0.095766,0.975331
300,No log,0.089538,0.977159
400,No log,0.078255,0.979899
500,0.151300,0.094211,0.978072
600,0.151300,0.115098,0.976245
700,0.151300,0.101402,0.976702
800,0.151300,0.113819,0.974418
900,0.151300,0.096674,0.979899
1000,0.015400,0.086190,0.981727


TrainOutput(global_step=1720, training_loss=0.049958703580290774, metrics={'train_runtime': 650.7117, 'train_samples_per_second': 84.292, 'train_steps_per_second': 2.643, 'total_flos': 1.44324188418048e+16, 'train_loss': 0.049958703580290774, 'epoch': 10.0})